In [7]:
from sklearn.ensemble import RandomForestRegressor

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.diagnostic import het_breuschpagan
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_log_error
from sklearn.tree import DecisionTreeRegressor, plot_tree


from statsmodels.tools.eval_measures import mse, rmse
from sklearn.linear_model import RidgeCV,LassoCV,ElasticNetCV


from statsmodels.graphics.regressionplots import plot_leverage_resid2
from statsmodels.stats.outliers_influence import variance_inflation_factor

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
def print_vif(x):
    """Utility for checking multicollinearity assumption
    
    :param x: input features to check using VIF. This is assumed to be a pandas.DataFrame
    :return: nothing is returned the VIFs are printed as a pandas series
    """
    # Silence numpy FutureWarning about .ptp
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        x = sm.add_constant(x)

    vifs = []
    for i in range(x.shape[1]):
        vif = variance_inflation_factor(x.values, i)
        vifs.append(vif)

    print("VIF results\n-------------------------------")
    print(pd.Series(vifs, index=x.columns))
    print("-------------------------------\n")
    
def eval_preds(y_true, y_pred,graph=False):
    error = y_true - y_pred

    rmse = np.sqrt((error ** 2).mean())
    mae = error.abs().mean()
    mape = (error / y_true).abs().mean()

    print(f"rmse {rmse}")
    print(f"mae {mae}")
    print(f"mape {mape}")
    
    if graph==True:
        line_pts = [y_true.min(), y_true.max()]
        plt.scatter(y_true, y_pred)
        plt.plot(line_pts, line_pts, c="red", ls="--", alpha=0.5)
        plt.xlabel("Actual")
        plt.ylabel("Fit")
        plt.show()


In [3]:
#Loading the dataset and manipulating the variables

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

#deal with missing values
drop_cols=[]
for col in df.columns:
    if df[col].isna().mean()>.4:
        drop_cols=drop_cols+[col]
df_clean=df.drop(columns=drop_cols).dropna()

#manipulate features, ones I don't use are commented out
df_clean['secondflrexists']=0
df_clean.loc[df['secondflrsf']>0,'secondflrexists']=1 #can prob drop 2nd fl sq ft
df_clean['yrsbltqared']=df_clean['yearbuilt']*df_clean['yearbuilt']
df_clean['agebuilt']=df_clean['yrsold']-df_clean['yearbuilt']
df_clean['ageremodeled']=df_clean['yrsold']-df_clean['yearremodadd']
df_clean['agebuiltsquared']=df_clean['agebuilt']*df_clean['agebuilt']
df_clean['agebuiltcubed']=df_clean['agebuilt']*df_clean['agebuilt']*df_clean['agebuilt']

# df_clean['logagebuilt']=np.log1p(df_clean['agebuilt'])
# df_clean['logageremodeled']=np.log1p(df_clean['ageremodeled'])

# df_clean['garageage']=df_clean['yrsold']-df_clean['garageyrblt']
# df_clean['garageagesquared']=df_clean['garageage']**2

####
df_clean['overallqualsquared']=df_clean['overallqual']*df_clean['overallqual']
df_clean['overallqualcubed']=df_clean['overallqual']*df_clean['overallqual']*df_clean['overallqual']
#df_clean['logqualsquared']=np.log(df_clean['overallqualsquared'])
# df_clean['logqual']=np.log(df_clean['overallqual'])
# df_clean['squaredlogqual']=df_clean['logqual']*df_clean['logqual']

df_clean['grlivareasquared']=df_clean['grlivarea']*df_clean['grlivarea']
# df_clean['loggrlivarea']=np.log(df_clean['grlivarea'])
# df_clean['squaredlogarea']=df_clean['loggrlivarea']*df_clean['loggrlivarea']


df_clean['ageremodeledsquared']=df_clean['ageremodeled']*df_clean['ageremodeled']
# df_clean['ageremodeledcubed']=df_clean['ageremodeled']*df_clean['ageremodeled']

df_clean['totalsf']=df_clean['totalbsmtsf']+df_clean['grlivarea']
# df_clean['qual_totalsf']=df_clean['totalsf']*df_clean['overallqual']

df_clean['lotareasquared']=df_clean['lotarea']*df_clean['lotarea']

#making partial,centralair and the neighborhood a binary variable
niceneighborhoods=['NridgHt', 'NoRidge', 'Somerst', 'Timber', 'Veenker', 'StoneBr']
goodneighborhoods=['ClearCr','Crawfor','CollgCr','Gilbert','Blmngtn','SawyerW','NWAmes']

df_clean['nicehood']=0
df_clean.loc[df_clean.neighborhood.isin(niceneighborhoods),'nicehood']=1
df_clean['hoodrank']=0
df_clean.loc[df_clean.neighborhood.isin(goodneighborhoods),'hoodrank']=1
df_clean.loc[df_clean.neighborhood.isin(niceneighborhoods),'hoodrank']=2
df_clean['goodhood']=0
df_clean.loc[df_clean.neighborhood.isin(goodneighborhoods),'goodhood']=1

#Making binary cats ints
df_clean['sale_partial']=0
df_clean.loc[df_clean.salecondition=='Partial','sale_partial']=1
df_clean['centralairint']=0
df_clean.loc[df_clean.centralair=='Y','centralairint']=1
# df_clean['pavedstreet']=0
# df_clean.loc[df_clean.salecondition=='Pave','pavedstreet']=1
df_clean['remodeled']=1
df_clean.loc[df_clean.agebuilt==df_clean.ageremodeled,'remodeled']=0
df_clean['pavedDW']=0
df_clean.loc[df_clean.paveddrive=='P','pavedDW']=1
df_clean['haspool']=0
df_clean.loc[df_clean.poolarea>0,'haspool']=1




#Making some interaction variables:
df_clean['nicehood_quality']=df_clean['overallqual']*df_clean['nicehood']
df_clean['goodhood_quality']=df_clean['overallqual']*df_clean['goodhood']
df_clean['nicehood_totalsf']=df_clean['totalsf']*df_clean['nicehood']
df_clean['goodhood_totalsf']=df_clean['totalsf']*df_clean['goodhood']

df_clean['totaloutside']=(df_clean['enclosedporch']+df_clean['wooddecksf']+ 
                          df_clean['openporchsf']+df_clean['threessnporch']+
                          df_clean['screenporch']#+df_clean['poolarea']
                         )
df_clean['totaloutside_quality']=df_clean['overallqual']*df_clean['totaloutside']

#one hot encoding neighborhood
#df_clean = pd.concat([df_clean,pd.get_dummies(df_clean.neighborhood, prefix="hood", drop_first=True)], axis=1)
df_clean = pd.concat([df_clean,pd.get_dummies(df_clean.kitchenqual, prefix="kitchen", drop_first=True)], axis=1)
df_clean = pd.concat([df_clean,pd.get_dummies(df_clean.bldgtype, prefix="buildingtype", drop_first=True)], axis=1)
#could try the other type of encoding instead

In [4]:
#Dropping outliers of high leveridge
df_clean=df_clean.drop([314,523,1298,452])

In [127]:
#Setting up the feature selection and OLS model

# Right hashtag indicates vars I'm iffy on
cat_cols=['nicehood','goodhood','centralairint','salepartial']
num_cols=['overallqual',
          'overallqualsquared',
          'overallqualcubed',
          'grlivarea',
          'totalbsmtsf',
          'garagearea',
          'lotarea',
          'lotareasquared',
          'ageremodeled',
          'ageremodeledsquared',
          'agebuilt',
          'agebuiltsquared',
          'bedroomabvgr']

X = df_clean[['overallqual',
              'overallqualsquared',#
              'overallqualcubed',#
              'grlivarea',
              'grlivareasquared',#
#               'loggrlivarea',#
               'totalbsmtsf',
               #'totalsf',#
               'garagearea', 
               'lotarea',
              'lotareasquared',
#               'nicehood',
#               'goodhood',
#               'hoodrank',#
              'neighborhood',
               'ageremodeled',
               'ageremodeledsquared',
#               'ageremodeledcubed',#
#               'logageremodeled',##
               'agebuilt',
               'agebuiltsquared',
#               'logagebuilt',##
               'sale_partial',#
              'centralairint',
#               'newsale',#
              'bedroomabvgr',#
#                'yrsold',
#               'nicehood_quality',
#               'goodhood_quality',
              'nicehood_totalsf',
              'goodhood_totalsf',
              'totaloutside',
#               'pavedDW',##
              'secondflrexists',#
#               'remodeled'##
#               'poolarea'##
              'haspool',
#               'totaloutside_quality'
#               'kitchen_Fa',#
#               'kitchen_Gd',#
#               'kitchen_TA',#
#               'pavedDW'##
              'buildingtype_2fmCon',
              'buildingtype_Duplex',
              'buildingtype_Twnhs',
              'buildingtype_TwnhsE',
              ]]
y = df_clean.saleprice
y_log=np.log(y)

In [128]:
from sklearn import tree
from category_encoders import LeaveOneOutEncoder
X_train, X_test, y_log_train, y_log_test=train_test_split(X,y_log,test_size=.2,random_state=1)

In [129]:
encoder = LeaveOneOutEncoder(cols=["neighborhood"])
X_train=encoder.fit_transform(X_train, y_train)
X_test=encoder.transform(X_test)

In [133]:
model = DecisionTreeRegressor(min_samples_leaf=15, max_depth=10)
model.fit(X_train, y_log_train)
# cross_val_score(model, X_train, y_log_train, cv=3)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=10,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=15, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [134]:
y_log_preds=model.predict(X_test)
y_log_train_preds=model.predict(X_train)

In [135]:
print(model.score(X_train, y_log_train))
print(model.score(X_test, y_log_test))


0.8745734314486844
0.8055594061386911


**Now, the RF model:**

In [154]:
from sklearn.ensemble import RandomForestRegressor
import time
start_time = time.time()

grid = {"max_depth": [5,6,8], "n_estimators": [200],'min_samples_leaf':[5, 10]}
model = GridSearchCV(
    RandomForestRegressor(),
    param_grid=grid,
    cv=4,
    #     scoring=make_scorer(f1_score),
#     scoring=make_scorer(roc_auc_score),
    verbose=1,
)
model.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))


Fitting 4 folds for each of 6 candidates, totalling 24 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   12.8s finished


--- 13.536211967468262 seconds ---


In [164]:
start_time = time.time()

model= RandomForestRegressor(min_samples_leaf=20, max_depth=8, n_estimators=50)
model.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.20643186569213867 seconds ---


In [165]:
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.8499934523064139
0.8395004289612634
